In [8]:
from ultralytics import YOLO
import cv2
import os
import torch
from torchvision.ops import nms


In [9]:
model = YOLO("C:/work/cs/pj/Thaifood_ObjectDetectionNClassification/datasets/food/runs/detect/train26/weights/best.pt")  # path to your trained weights


In [3]:
# crop 1 จานได้
def crop_from_results(results, image_paths, output_dir='cropped', conf_threshold=0.5, target_class=1):
    os.makedirs(output_dir, exist_ok=True)

    for idx, result in enumerate(results):
        image_path = image_paths[idx]
        image = cv2.imread(image_path)

        if image is None:
            print(f"❌ Failed to read {image_path}")
            continue

        # Extract boxes, class IDs, and confidences
        try:
            boxes = result.boxes.xyxy.cpu().numpy()  # [x1, y1, x2, y2, conf]
            class_ids = result.boxes.cls.cpu().numpy()
            confidences = result.boxes.conf.cpu().numpy()
        except AttributeError:
            print(f"❌ result[{idx}] is missing .boxes or attributes")
            continue

        crop_count = 0
        for i, (box, cls_id, conf) in enumerate(zip(boxes, class_ids, confidences)):
            if conf < conf_threshold:
                continue
            if int(cls_id) != target_class:
                continue  # skip other classes

            x1, y1, x2, y2 = map(int, box[:4])
            cropped = image[y1:y2, x1:x2]

            basename = os.path.basename(image_path).split('.')[0]
            out_path = os.path.join(output_dir, f"{basename}_class{target_class}_crop{crop_count+1}.jpg")
            cv2.imwrite(out_path, cropped)
            print(f"✅ Saved: {out_path}")
            crop_count += 1


In [10]:
def crop_all_dishes_with_torch_nms(results, image_paths, output_dir='cropped', conf_threshold=0.3, iou_threshold=0.5, class_names=None):
    os.makedirs(output_dir, exist_ok=True)

    for idx, result in enumerate(results):
        image = result.orig_img
        if image is None:
            print(f"❌ Failed to read image at index {idx}")
            continue

        try:
            boxes = result.boxes.xyxy  # torch.Tensor [N, 4]
            scores = result.boxes.conf  # torch.Tensor [N]
            class_ids = result.boxes.cls.int()  # torch.Tensor [N]
        except AttributeError:
            print(f"❌ result[{idx}] missing .boxes attributes")
            continue

        basename = os.path.splitext(os.path.basename(image_paths[idx]))[0]

        for class_id in class_ids.unique():
            cls_mask = class_ids == class_id
            cls_boxes = boxes[cls_mask]
            cls_scores = scores[cls_mask]

            # Apply NMS
            keep = nms(cls_boxes, cls_scores, iou_threshold)

            for crop_count, i in enumerate(keep):
                if cls_scores[i] < conf_threshold:
                    continue

                x1, y1, x2, y2 = map(int, cls_boxes[i].tolist())
                crop = image[y1:y2, x1:x2]

                class_name = f"class{class_id}"
                if class_names and class_id < len(class_names):
                    class_name = class_names[class_id]

                save_path = os.path.join(output_dir, f"{basename}_{class_name}_crop{crop_count+1}.jpg")
                cv2.imwrite(save_path, crop)
                print(f"✅ Saved: {save_path}")


In [12]:
# แก้pathก้ด้วยนะ ของกุใส่.ละมันหาไม่เจอ
image_dir = r"C:\work\cs\pj\Thaifood_ObjectDetectionNClassification\datasets\food\traintest"
# อันนี้กุลองบางรูปนะ
# image_names = ["plate1.jpg", "plate2.jpg", "plate3.jpg", "plate4.jpg", "plate5.jpg"]
image_names = ["plate11.jpg"]

image_paths = [os.path.join(image_dir, name) for name in image_names]

# Make sure 'results' is already created with model(image_paths)
# Then call:
results = [model(os.path.join(image_dir, name))[0] for name in image_names]
# crop_from_results(results, image_paths, target_class=1)  # dish2
class_names = ["dish1", "dish2", "plate"]
# crop_all_dishes_from_results(results, image_paths, class_names=class_names)
crop_all_dishes_with_torch_nms(
    results=results,
    image_paths=image_paths,
    output_dir="cropped",
    conf_threshold=0.3,
    iou_threshold=0.5,
    class_names=class_names
)


image 1/1 C:\work\cs\pj\Thaifood_ObjectDetectionNClassification\datasets\food\traintest\plate11.jpg: 640x512 11 dish1s, 220.5ms
Speed: 5.2ms preprocess, 220.5ms inference, 3.4ms postprocess per image at shape (1, 3, 640, 512)
✅ Saved: cropped\plate11_dish1_crop1.jpg
✅ Saved: cropped\plate11_dish1_crop2.jpg
✅ Saved: cropped\plate11_dish1_crop3.jpg
✅ Saved: cropped\plate11_dish1_crop4.jpg
✅ Saved: cropped\plate11_dish1_crop5.jpg
✅ Saved: cropped\plate11_dish1_crop6.jpg
✅ Saved: cropped\plate11_dish1_crop7.jpg
✅ Saved: cropped\plate11_dish1_crop8.jpg
✅ Saved: cropped\plate11_dish1_crop9.jpg
✅ Saved: cropped\plate11_dish1_crop10.jpg
✅ Saved: cropped\plate11_dish1_crop11.jpg


In [7]:
results = model(image_dir)
print(type(results))
print(dir(results))


image 1/20 C:\work\cs\pj\Thaifood_ObjectDetectionNClassification\datasets\food\traintest\output_plate1.jpg: 448x640 1 dish2, 54.6ms
image 2/20 C:\work\cs\pj\Thaifood_ObjectDetectionNClassification\datasets\food\traintest\output_plate2.jpg: 448x640 1 dish1, 2 dish2s, 27.8ms
image 3/20 C:\work\cs\pj\Thaifood_ObjectDetectionNClassification\datasets\food\traintest\output_plate3.jpg: 448x640 1 dish2, 27.9ms
image 4/20 C:\work\cs\pj\Thaifood_ObjectDetectionNClassification\datasets\food\traintest\output_plate4.jpg: 448x640 1 dish2, 28.6ms
image 5/20 C:\work\cs\pj\Thaifood_ObjectDetectionNClassification\datasets\food\traintest\output_plate5.jpg: 480x640 1 dish2, 34.0ms
image 6/20 C:\work\cs\pj\Thaifood_ObjectDetectionNClassification\datasets\food\traintest\output_plate6.jpg: 448x640 1 dish1, 12 dish2s, 28.6ms
image 7/20 C:\work\cs\pj\Thaifood_ObjectDetectionNClassification\datasets\food\traintest\output_plate7.jpg: 384x640 1 dish1, 1 dish2, 62.4ms
image 8/20 C:\work\cs\pj\Thaifood_ObjectDetec

In [ ]:
# ไม่น่าต้องใละ
def yolo_to_xyxy(xc, yc, w, h, img_w, img_h):
    x1 = int((xc - w / 2) * img_w)
    y1 = int((yc - h / 2) * img_h)
    x2 = int((xc + w / 2) * img_w)
    y2 = int((yc + h / 2) * img_h)
    return [x1, y1, x2, y2]